<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/mcp_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI MCP Agent

[OpenAI MCP Client/Server in Colab](https://docs.google.com/document/d/1jgTf6iS1QgEgxNK06nH7lsMjQ4UT9VUpkx6pswICy4o/edit?tab=t.0#heading=h.nevkjb3cgate)

In [ ]:
!pip install openai mcp python-dotenv anyio
!pip install openai-agents

In [ ]:
%%writefile my_mcp_server.py

import asyncio
from mcp.server import Server
from mcp.types import Tool, ToolResult

server = Server("colab-server")

@server.tool()
async def add_numbers(a: int, b: int) -> ToolResult:
    return ToolResult(content=f"The sum is {a + b}")

async def main():
    await server.run_stdio()

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import asyncio
import subprocess

server_process = await asyncio.create_subprocess_exec(
    "python3", "my_mcp_server.py",
    stdin=asyncio.subprocess.PIPE,
    stdout=asyncio.subprocess.PIPE,
    stderr=asyncio.subprocess.PIPE
)

print("MCP server started in Colab")

In [ ]:
import os
import warnings
from google.colab import userdata

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

# Initialize API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

print("The OPENAI_API_KEY was retreived from the Colab environment")

In [ ]:
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio, MCPServerStdioParams
from IPython.display import display, Markdown

async def main(input: str):

    name: str = "colab-server"
    params: MCPServerStdioParams = {"command": "python3", "args": ["my_mcp_server.py"]}

    async with MCPServerStdio(
        name=name,
        params=params,
        cache_tools_list=True,
        max_retry_attempts=3,
    ) as server:
        agent = Agent(
            name="Assistant",
            instructions="You are a helpful assistant that can add numbers together using a MCP tool.",
            mcp_servers=[server]
        )

        result = await Runner.run(agent, input=input)
        print(result.final_output)


# Run the async function
await main("Add 5 and 3")